### Association Rule Mining
 - 연관 분석

In [70]:
import os
import mlxtend
import pandas as pd
import numpy as np

In [242]:
path = '../archive/data/movielens/'

ratings_df = pd.read_csv(os.path.join(path, 'ratings.csv'), encoding='utf-8')
movies_df = pd.read_csv(os.path.join(path, 'movies.csv'), encoding='utf-8')

In [243]:
viewList = []
for group in ratings_df.groupby('userId')['movieId']:
    viewList.append(group[1].values)

In [244]:
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(viewList).transform(viewList)
df = pd.DataFrame(te_ary, columns=te.columns_)

### Apriori & FP-growth

In [247]:
from mlxtend.frequent_patterns import apriori , fpgrowth , association_rules

#apriori_df = apriori(df, min_support=0.1 , use_colnames=True , max_len=4)
fpgrowth_df = fpgrowth(df, min_support=0.1 , use_colnames=True , max_len=4)

In [248]:
association_rules(fpgrowth_df , metric='confidence',min_threshold=0.5 , support_only = False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(296),(356),0.503279,0.539344,0.377049,0.749186,1.389068,0.105609,1.836640
1,(356),(296),0.539344,0.503279,0.377049,0.699088,1.389068,0.105609,1.650720
2,(296),(318),0.503279,0.519672,0.363934,0.723127,1.391506,0.102395,1.734831
3,(318),(296),0.519672,0.503279,0.363934,0.700315,1.391506,0.102395,1.657481
4,"(296, 356)",(318),0.377049,0.519672,0.293443,0.778261,1.497600,0.097501,2.166185
...,...,...,...,...,...,...,...,...,...
404191,(1393),(780),0.139344,0.331148,0.101639,0.729412,2.202679,0.055496,2.471846
404192,(1393),(260),0.139344,0.411475,0.106557,0.764706,1.858449,0.049221,2.501230
404193,(1393),(593),0.139344,0.457377,0.104918,0.752941,1.646215,0.041185,2.196331
404194,(1393),(356),0.139344,0.539344,0.114754,0.823529,1.526909,0.039600,2.610383


In [236]:
# apriori_df = apriori_df.sort_values('support' ,ascending=False)
# apriori_df['itemsets'] = apriori_df['itemsets'].map(lambda x : list(x))

In [249]:
fpgrowth_df = fpgrowth_df.sort_values('support' ,ascending=False)
fpgrowth_df['itemsets'] = fpgrowth_df['itemsets'].map(lambda x : list(x))

In [250]:
fpgrowth_df['size'] = fpgrowth_df['itemsets'].map(lambda x : len(x))

In [251]:
rule_df = fpgrowth_df[fpgrowth_df['size'] == 2]

In [252]:
rule_df['item_1'] = rule_df['itemsets'].map(lambda x : x[0])
rule_df['item_2'] = rule_df['itemsets'].map(lambda x : x[1])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [253]:
rule_df

,support,itemsets,size,item_1,item_2
32554,0.378689,"[356, 318]",2,356,318
328,0.377049,"[296, 356]",2,296,356
329,0.363934,"[296, 318]",2,296,318
331,0.339344,"[296, 593]",2,296,593
332,0.326230,"[593, 356]",2,593,356
...,...,...,...,...,...
56656,0.100000,"[4993, 6333]",2,4993,6333
56658,0.100000,"[8665, 7153]",2,8665,7153
56659,0.100000,"[8665, 356]",2,8665,356
56661,0.100000,"[8665, 2959]",2,8665,2959


In [301]:
movieId_size = pd.DataFrame(ratings_df.groupby('movieId').size())

In [302]:
movieId_size = movieId_size.reset_index()
movieId_size.columns = ['item_1' , 'count_1']

In [303]:
rule_df = pd.merge(rule_df , movieId_size , how='inner',on='item_1')

In [305]:
movieId_size.columns = ['item_2' , 'count_2']

In [306]:
rule_df = pd.merge(rule_df , movieId_size , how='inner',on='item_2')

In [319]:
rule_df['sum_count'] = rule_df['count_1'] + rule_df['count_2']
rule_df

,support,itemsets,size,item_1,item_2,count_1,count_2,sum_count
0,0.378689,"[356, 318]",2,356,318,329,317,646
1,0.363934,"[296, 318]",2,296,318,307,317,624
2,0.326230,"[593, 318]",2,593,318,279,317,596
3,0.252459,"[480, 318]",2,480,318,238,317,555
4,0.296721,"[2571, 318]",2,2571,318,278,317,595
...,...,...,...,...,...,...,...,...
5056,0.134426,"[288, 296]",2,288,296,92,307,399
5057,0.109836,"[1784, 296]",2,1784,296,96,307,403
5058,0.103279,"[780, 95]",2,780,95,202,84,286
5059,0.101639,"[780, 141]",2,780,141,202,86,288
